# Nemo parametric hull shapes

----
Diego Montero, Fernando Valentini, Gustavo Violato <br>
First Release: Jul. 2016

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy.interpolate as interpolate
from mayavi import mlab

%gui qt

import hull_geom as geo
import hull_dynamics as dyn

In [3]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Range1d

In [4]:
output_notebook()

Loading BokehJS ...

In [5]:
# user defined variables
L      = 2.30;       # hull length (LOA) in m
alt    = 0.005;       # Height of top
pontal = 0.23;       # Draft
mboca  = 0.29/2      # Half-beam
w      = 1.35        # Separation

# Speeds for take-off draft study
V_TO = [1.0,1.25,1.5,1.75,2.0,2.25,2.5,2.75,3.0,3.25]

# Define the total mass and CG position
mass = 87+14.
cgZ = np.array([0.075, 0., 0.2])

In [5]:
cap = lambda x: -1*geo.trapz(x,alt/L,perc=0.99)
kee = lambda x: geo.trapz(x,pontal/L,perc=0.96)
stt = np.linspace(-L/2,L/2,100)
top_cl = []
bot_cl = []
for s in stt:
    top_cl.append(cap(2*s/L)*L)
    bot_cl.append(kee(2*s/L)*L)

p = figure(width=900,height=300,toolbar_location='above')
p.y_range = Range1d(-0.6,0.4)
p.x_range = Range1d(-1.5,1.5)
p.line(stt,top_cl)
p.line(stt,bot_cl)
show(p)

In [6]:
class BuoyShape():
    def __init__(self,length,depth,width,height):
        self.length = length
        self.keel   = lambda x: geo.trapz(x,depth/length,perc=0.96)
        self.waterline   = lambda x: -geo.normal_pol(x,width/length,4)+0.01*np.sin(np.pi*x)
        self.bottom   = lambda x,b,c: geo.wigley1_cross_R(x,b,c,rb=0.04)
        self.cap    = lambda x: -geo.trapz(x,height/length,perc=0.99)
        self.top   = lambda x,b,c: -geo.trapz(x,height,perc=0.99)
        
shape = BuoyShape(L,pontal,mboca,alt)

In [7]:
# Equilibrium calculation

(phi, theta, dz) = dyn.equilibrium(mass,cgZ,shape,w,verbose=1)
print "Found equilibirum position. Equilibrium variables are:"
print "Phi={:.1f}".format(phi)+"deg"
print "Theta={:.1f}".format(theta)+"deg"
print "Dz={:.0f}".format(dz*1000)+"mm"

Solution: [  5.24406382e-07   5.06388266e-02   5.90633374e-02]
Residual: [  8.08302675e+00  -3.82878598e-05   7.59764481e-02]
Solution: [  3.35596996e-10   5.18776672e-02   5.98921675e-02]
Residual: [  7.80005384e-03  -2.44615990e-08  -5.62499375e-03]
Solution: [ -1.06063871e-11   5.17608001e-02   5.98928183e-02]
Residual: [  1.02522790e-08   7.73070497e-10  -6.07114782e-09]
Found equilibirum position. Equilibrium variables are:
Phi=-0.0deg
Theta=0.1deg
Dz=60mm


In [8]:
def plot_scene(shape,w,angles,position,cgPos_e):
    # Calculate buoy shape coordinates in its intrinsic coordinate
    # system
    csB_coords_b = geo.getShapePoints(shape,'bottom',80, 60)
    csT_coords_b = geo.getShapePoints(shape,'top',80, 60)

    # Calculate rotated and translated coordinates for left and right
    # buoys
    bl_pos = np.array([0.,
                       -w/2*np.cos(np.radians(angles[0])),
                       -w/2*np.sin(np.radians(angles[0]))])
    csLB_e = dyn.buoy_to_earth(csB_coords_b,angles,position+bl_pos)
    csRB_e = dyn.buoy_to_earth(csB_coords_b,angles,position-bl_pos)
    csLT_e = dyn.buoy_to_earth(csT_coords_b,angles,position+bl_pos)
    csRT_e = dyn.buoy_to_earth(csT_coords_b,angles,position-bl_pos)
    
    xlb = csLB_e[0,:].reshape((80,119))
    ylb = csLB_e[1,:].reshape((80,119))
    zlb = csLB_e[2,:].reshape((80,119))

    xrb = csRB_e[0,:].reshape((80,119))
    yrb = csRB_e[1,:].reshape((80,119))
    zrb = csRB_e[2,:].reshape((80,119))

    xlt = csLT_e[0,:].reshape((80,119))
    ylt = csLT_e[1,:].reshape((80,119))
    zlt = csLT_e[2,:].reshape((80,119))

    xrt = csRT_e[0,:].reshape((80,119))
    yrt = csRT_e[1,:].reshape((80,119))
    zrt = csRT_e[2,:].reshape((80,119))

    slb = mlab.mesh(xlb,ylb,zlb, color=(139./255.,139./255.,0))
    srb = mlab.mesh(xrb,yrb,zrb, color=(139./255.,139./255.,0))
    slt = mlab.mesh(xlt,ylt,zlt, color=(139./255.,139./255.,0))
    srt = mlab.mesh(xrt,yrt,zrt, color=(139./255.,139./255.,0))

    wtp = mlab.surf(np.array([-2.,2.]),np.array([-2.,2.]),
                    np.zeros((2,2)),color=(0,0,1),opacity=0.3)
    
    mlab.points3d(cgPos_e[0],cgPos_e[1],cgPos_e[2],color=(1,0,0),
                 scale_factor=0.075)

    mlab.show()

In [ ]:
angles = np.array([phi,theta,0])
position = np.array([0.,0.,dz])
cgPos_e = cgZ+position
plot_scene(shape,w,angles,position,cgPos_e)

In [9]:
HULL_NAME = 'nemohull_a_trap2.txt' # nemohull_wig.txt
mlt_pnts = geo.geom_to_mlt(shape, 0, 81, 61)
np.savetxt(HULL_NAME,-1*mlt_pnts,fmt='%.6f',delimiter=',')

In [10]:
print '1-Hull Volume: {:.5f} m^3'.format(-1*geo.volume(shape.length,
                                                           shape.keel,
                                                           shape.waterline,
                                                           shape.bottom))

TypeError: <lambda>() takes exactly 3 arguments (1 given)

In [11]:
wl  = lambda x: -geo.normal_pol(x,mboca/L,4)+0.01*np.sin(np.pi*x)
stt = np.linspace(-L/2,L/2,100)
wl_cl = []
for s in stt:
    wl_cl.append(wl(2*s/L)*L)

p = figure(width=900,height=200,toolbar_location='above')
p.y_range = Range1d(-0.3,0.3)
p.x_range = Range1d(-1.5,1.5)
p.line(stt,wl_cl)
p.line(stt,-1*np.array(wl_cl))
show(p)

In [12]:
from scipy.optimize import fsolve

def d_wl(x):
    return -4*mboca/L*x**3 + 0.01*np.pi*np.cos(np.pi*x)

def func_dx1(dx1, *args):
    x0 = args[0]
    t  = args[1]
    return wl(x0+dx1) - (wl(x0)+t) - dx1*d_wl(x0+dx1)

def func_dx2(dx2, *args):
    x0  = args[0]
    t   = args[1]
    dx1 = args[2]
    return wl(x0+dx1+dx2)+tol-wl(x0+dx1)-dx2*d_wl(x0+dx1)

def linepntsfromsol(sol_l, tol):
    xt = np.array([sol_l[0], np.sum(sol_l)])*L/2
    yt = np.array([wl(sol_l[0])+tol, wl(np.sum(sol_l))+tol])*L
    return xt, yt

In [13]:
e = 0.003*2/L
stt_start = 2*(-L/2+0.02*L+e)/L
dx_guess = 0.1*2/L
tol = 0.004/L

sol1 = []
stt_c = stt_start
for _ in xrange(2):
    dx1, infodict1, ier1, mesg1 = fsolve(func_dx1, dx_guess, args=(stt_c,tol), full_output=1)
    if ier1==1:
        dx2, infodict2, ier2, mesg2 = fsolve(func_dx2, dx_guess, args=(stt_c,tol,dx1), full_output=1)
    else:
        print "dx1 not found"
    if ier2!=1:
        print "dx2 not found"
    sol1.append([stt_c, dx1, dx2])
    stt_c = stt_c+dx1+dx2+e

sol2 = []
stt_start = 0
stt_c = stt_start
for _ in xrange(4):
    dx1, infodict1, ier1, mesg1 = fsolve(func_dx1, dx_guess, args=(stt_c,tol), full_output=1)
    if ier1==1:
        dx2, infodict2, ier2, mesg2 = fsolve(func_dx2, dx_guess, args=(stt_c,tol,dx1), full_output=1)
    else:
        print "dx1 not found"
    if ier2!=1:
        print "dx2 not found"
    sol2.append([stt_c, dx1, dx2])
    stt_c = stt_c+dx1+dx2+e
    
mdl_start = np.sum(sol1[-1])+e
mdl_dx1 = 0
mdl_dx2 = sol2[0][0]-e-mdl_start
m_stt = [[mdl_start, mdl_dx1, mdl_dx2]]    

stt_end = 2*(L/2-0.02*L-e)/L
if np.sum(sol2[-1])>stt_end:
    print "End reached"
    dx2_end = stt_end - sol2[-1][0]
    sol2[-1] = [sol2[-1][0], 0, dx2_end]

sol = np.array(sol1 + m_stt + sol2)

In [14]:
linepntsfromsol(sol[0,:],tol=tol)

(array([-1.1013913, -0.9181925]), array([ 0.02395946,  0.0764626 ]))

In [15]:
p = figure(width=900,height=400,toolbar_location='above')
p.y_range = Range1d(-0.3,0.3)
p.x_range = Range1d(-1.5,1.5)
p.line(stt,wl_cl)
p.line(stt,-1*np.array(wl_cl))
for i,l in enumerate(sol):
    if i==2:
        xt, yt = linepntsfromsol(l,tol=0.004/L)
    else:
        xt, yt = linepntsfromsol(l,tol=tol)
    p.line(xt,yt,color='red')
    p.line(xt,-1*np.array(yt),color='red')

show(p)

In [18]:
xt,yt = linepntsfromsol(sol[1,:],tol=tol)
cs_b = geo.cross_section(xt[0]+shape.length/2,shape.length,shape.keel,shape.waterline,shape.bottom,100)
cs_b[0,:] = cs_b[0,:]*yt[0]/cs_b[0,-1]
cs_b2 = geo.cross_section(xt[1]+shape.length/2,shape.length,shape.keel,shape.waterline,shape.bottom,100)
cs_b2[0,:] = cs_b2[0,:]*yt[1]/cs_b2[0,-1]
cs_b_f = np.empty((cs_b.shape[0],cs_b.shape[1]*2))
cs_b_f[:,:cs_b.shape[1]] = cs_b[:,::-1]
cs_b_f[:,cs_b.shape[1]:] = cs_b
cs_b_f[0,cs_b.shape[1]:] *= -1

cs_b_f2 = np.empty((cs_b2.shape[0],cs_b2.shape[1]*2))
cs_b_f2[:,:cs_b2.shape[1]] = cs_b2[:,::-1]
cs_b_f2[:,cs_b2.shape[1]:] = cs_b2
cs_b_f2[0,cs_b2.shape[1]:] *= -1

p = figure(width=520, height=500)
p.y_range = Range1d(-250,50)
p.x_range = Range1d(-125,125)
p.line(cs_b_f[0,:]*1000,cs_b_f[1,:]*1000)
p.line(cs_b_f2[0,:]*1000,cs_b_f2[1,:]*1000,color='red')
show(p)

In [16]:
COORDS_DIR = './cross_sections'

sections = range(100)
coords   = []

for s in sections:
    coords.append(np.loadtxt(COORDS_DIR + '/cs_{:02d}.sldcrv'.format(s)))

    
p = figure(width=520, height=500)
for sp in [0,4,10,15,30]:
    p.line(coords[sp][:,0],coords[sp][:,1])
show(p)